In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "ai-forever/mGPT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

In [2]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"], 
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    fan_in_fan_out=True,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 4,325,376 || all params: 1,421,922,304 || trainable%: 0.3042


In [3]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="romania_dataset.json")
dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)

def tokenize_function(examples):
    texts = [f"{title}\n{content}" for title, content in zip(examples["title"], examples["text"])]
    
    tokenized = tokenizer(
        texts, 
        padding="max_length", 
        truncation=True, 
        max_length=512,
        return_tensors="pt"
    )

    tokenized["labels"] = tokenized["input_ids"].clone()
    tokenized["labels"][tokenized["input_ids"] == tokenizer.pad_token_id] = -100
    
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["title", "text"])

In [4]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./mgpt-lora-finetuned_romania_04",
    per_device_train_batch_size=4,
    num_train_epochs=4,
    save_strategy="steps",
    save_steps=2500,
    eval_strategy="steps",
    save_total_limit=1,
    logging_steps=2500,
    learning_rate=0.0001,
    fp16=True,
    report_to="none",
    label_names=['labels'],
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False
)

In [5]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
2500,1.669200,1.548174
5000,1.598200,1.533960
7500,1.577400,1.527619
10000,1.570400,1.519004
12500,1.549600,1.517801
15000,1.547000,1.514853
17500,1.535600,1.511870
20000,1.541500,1.513373


TrainOutput(global_step=20736, training_loss=1.5720080328576358, metrics={'train_runtime': 10443.5342, 'train_samples_per_second': 7.942, 'train_steps_per_second': 1.986, 'total_flos': 3.09058867298304e+17, 'train_loss': 1.5720080328576358, 'epoch': 4.0})

In [6]:
model.save_pretrained("mgpt-lora-adapter_romania_04")


In [7]:
tokenizer.save_pretrained("mgpt-lora-adapter_romania_04_tok")

('mgpt-lora-adapter_romania_04_tok/tokenizer_config.json',
 'mgpt-lora-adapter_romania_04_tok/special_tokens_map.json',
 'mgpt-lora-adapter_romania_04_tok/vocab.json',
 'mgpt-lora-adapter_romania_04_tok/merges.txt',
 'mgpt-lora-adapter_romania_04_tok/added_tokens.json',
 'mgpt-lora-adapter_romania_04_tok/tokenizer.json')